In [21]:
import pandas
import numpy as np

In [24]:
obligations = pandas.read_csv("../Data/obligations.csv")
summary = obligations.groupby(["DateScraped", "CitationNumber"])

def clean(data):
    return float(data.strip().replace("$",""))

differences = {"DateScraped": [],
               "CitationNumber": [],
               "TotalOriginalAmount": [],
               "TotalBalance": [],
               "BalanceDifference": []}

for group, data in summary:
    #print(group)
    #print(data)
    for i in range(len(data)):
        OriginalAmounts = data["OriginalAmount"]
        original = np.zeros(len(OriginalAmounts))
        for j in range(len(OriginalAmounts)):
            original[j] = clean(OriginalAmounts.iloc[j,])
        Balances = data["Balance"]
        balances = np.zeros(len(Balances))
        for j in range(len(Balances)):
            balances[j] = clean(Balances.iloc[j,])
        
        original_total = np.sum(original)
        balance_total = np.sum(balances)
        difference = original_total-balance_total
        
        differences["DateScraped"].append(group[0])
        differences["CitationNumber"].append(group[1])
        differences["TotalOriginalAmount"].append(original_total)
        differences["TotalBalance"].append(balance_total)
        differences["BalanceDifference"].append(difference)
        
        #print(np.sum(original) - np.sum(balances))
        
differences_data = pandas.DataFrame(differences)
differences_data

,DateScraped,CitationNumber,TotalOriginalAmount,TotalBalance,BalanceDifference
0,2023-03-18,2100935015,139.0,0.0,139.0
1,2023-03-18,2200192913,237.0,0.0,237.0
2,2023-03-18,2A0084843,191.0,0.0,191.0
3,2023-03-18,2A0084843,191.0,0.0,191.0
4,2023-03-18,2A0084843,191.0,0.0,191.0
...,...,...,...,...,...
252,2023-07-06,9Z0828346,191.0,191.0,0.0
253,2023-07-06,9Z0828346,191.0,191.0,0.0
254,2023-07-06,9Z0828346,191.0,191.0,0.0
255,2023-07-06,9Z0828346,191.0,191.0,0.0


In [12]:
differences_summary1 = differences_data.groupby("CitationNumber")
differences_summary2 = differences_data.groupby(["CitationNumber", "BalanceDifference"])

citation_balance_changes = []
for group, data in differences_summary1:
    balance_changes = data["BalanceDifference"].nunique()-1
    citation_balance_changes.append([str(group), balance_changes])

citation_balance_changes = pandas.DataFrame(citation_balance_changes,
                                           columns=["CitationNumber", "TotalBalanceChanges"])

balance_changes = {
    "CitationNumber": [],
    "TotalOriginalAmount": [],
    "TotalBalanceChanges": [],
    "BalanceChanges": [],
    "DateChangeWasCaptured": []
}

for group, data in differences_summary2:
    balance_changes["CitationNumber"].append(str(group[0]))
    balance_charges = citation_balance_changes[citation_balance_changes["CitationNumber"] == str(group[0])]
    balance_changes["TotalBalanceChanges"].append(
        balance_charges.iloc[0,1]
    )

balance_changes

{'CitationNumber': ['2100935015',
  '2200192913',
  '2300538791',
  '2300585594',
  '2300665354',
  '2A0084843',
  '2A0365587',
  '2A0530631',
  '2A0530631',
  '73237039',
  '73237039',
  '9Z0828346'],
 'TotalOriginalAmount': [],
 'TotalBalanceChanges': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
 'BalanceChanges': [],
 'DateChangeWasCaptured': []}

In [25]:
differences_data = differences_data.drop_duplicates().reset_index(drop=True)
differences_data["BalanceChange"] = np.zeros(len(differences_data))
differences_data["OriginalAmountChange"] = np.zeros(len(differences_data))
differences_summary = differences_data.groupby("CitationNumber")
for group, data in differences_summary:
    data_reindexed = pandas.DataFrame(data).reset_index(drop=True)
    amounts = data_reindexed["TotalOriginalAmount"].values
    amount0 = data_reindexed.at[0,"TotalOriginalAmount"]
    balance_differences = data_reindexed["BalanceDifference"].values
    balance0 = data_reindexed.at[0, "BalanceDifference"]
    for i in range(len(amounts)):
        if amount0 != amounts[i]:
            amount0 = amounts[i]
            data_reindexed.at[i,"OriginalAmountChange"] = True
        else:
            data_reindexed.at[i,"OriginalAmountChange"] = False
        if balance0 != balance_differences[i]:
            balance0 = balance_differences[i]
            data_reindexed.at[i,"BalanceChange"] = True
        else:
            data_reindexed.at[i,"BalanceChange"] = False
        
#     print(differences_data.loc[data.index, "OriginalAmountChange"])
#     print(data_reindexed["OriginalAmountChange"].values)
    
    differences_data.loc[data.index, "OriginalAmountChange"] = data_reindexed["OriginalAmountChange"].values
    differences_data.loc[data.index, "BalanceChange"] = data_reindexed["BalanceChange"].values

differences_data

,DateScraped,CitationNumber,TotalOriginalAmount,TotalBalance,BalanceDifference,BalanceChange,OriginalAmountChange
0,2023-03-18,2100935015,139.0,0.0,139.0,False,False
1,2023-03-18,2200192913,237.0,0.0,237.0,False,False
2,2023-03-18,2A0084843,191.0,0.0,191.0,False,False
3,2023-03-18,2A0365587,191.0,0.0,191.0,False,False
4,2023-03-18,2A0530631,128.0,128.0,0.0,False,False
...,...,...,...,...,...,...,...
65,2023-07-06,2A0084843,191.0,0.0,191.0,False,False
66,2023-07-06,2A0365587,191.0,0.0,191.0,False,False
67,2023-07-06,2A0530631,272.0,231.0,41.0,False,False
68,2023-07-06,73237039,72.0,0.0,72.0,False,False


In [26]:
differences_data.sort_values(by=['CitationNumber', 'DateScraped'], ascending=False, inplace=True)
differences_data = differences_data.reset_index(drop=True)
differences_data

,DateScraped,CitationNumber,TotalOriginalAmount,TotalBalance,BalanceDifference,BalanceChange,OriginalAmountChange
0,2023-07-06,9Z0828346,191.0,191.0,0.0,False,False
1,2023-07-05,9Z0828346,191.0,191.0,0.0,False,False
2,2023-07-02,9Z0828346,191.0,191.0,0.0,False,False
3,2023-06-28,9Z0828346,191.0,191.0,0.0,False,False
4,2023-06-27,9Z0828346,191.0,191.0,0.0,False,False
...,...,...,...,...,...,...,...
65,2023-07-02,2100935015,139.0,0.0,139.0,False,False
66,2023-06-28,2100935015,139.0,0.0,139.0,False,False
67,2023-06-27,2100935015,139.0,0.0,139.0,False,False
68,2023-06-09,2100935015,139.0,0.0,139.0,False,False


In [27]:
citations = differences_data.groupby("CitationNumber")
ChangeTotals = {
    "CitationNumber": [],
    "TotalChanges": []
}
for citation, data in citations:
    ChangeTotals["CitationNumber"].append(citation)
    data = pandas.DataFrame(data)
    ChangeTotals["TotalChanges"].append(np.sum(data["BalanceChange"].values.astype(int)) + np.sum(data["OriginalAmountChange"].values.astype(int)))
ChangeTotals = pandas.DataFrame(ChangeTotals)
ChangeTotals.sort_values(by=['TotalChanges', 'CitationNumber'], ascending=False, inplace=True)
ChangeTotals.reset_index(drop=True)

,CitationNumber,TotalChanges
0,2A0530631,2
1,9Z0828346,1
2,73237039,1
3,2A0365587,0
4,2A0084843,0
5,2300665354,0
6,2300585594,0
7,2300538791,0
8,2200192913,0
9,2100935015,0


In [28]:
import analysis
analysis.obligations_summary()

,DateScraped,CitationNumber,TotalOriginalAmount,TotalBalance,BalanceDifference,BalanceChange,OriginalAmountChange
0,2023-07-06,9Z0828346,191.0,191.0,0.0,False,False
1,2023-07-05,9Z0828346,191.0,191.0,0.0,False,False
2,2023-07-02,9Z0828346,191.0,191.0,0.0,False,False
3,2023-06-28,9Z0828346,191.0,191.0,0.0,False,False
4,2023-06-27,9Z0828346,191.0,191.0,0.0,False,False
...,...,...,...,...,...,...,...
65,2023-07-02,2100935015,139.0,0.0,139.0,False,False
66,2023-06-28,2100935015,139.0,0.0,139.0,False,False
67,2023-06-27,2100935015,139.0,0.0,139.0,False,False
68,2023-06-09,2100935015,139.0,0.0,139.0,False,False


In [30]:
case_details = pandas.read_csv("../Data/case_details.csv")
summary = case_details.groupby(["DateScraped", "CitationNumber"])

  CitationNumber DateScraped CaseStatus      DefenseAttorney
0      2A0084843  2023-03-18       OPEN  VINCENT Q. WILLIAMS
  CitationNumber DateScraped CaseStatus      DefenseAttorney
0      2A0365587  2023-03-18       OPEN  VINCENT Q. WILLIAMS
  CitationNumber DateScraped CaseStatus         DefenseAttorney
0      2A0530631  2023-03-18       OPEN  MITCHELL REUBEN GREENE
  CitationNumber DateScraped CaseStatus DefenseAttorney
0      9Z0828346  2023-03-18       OPEN             NaN
  CitationNumber DateScraped CaseStatus      DefenseAttorney
0      2A0084843  2023-06-09       OPEN  VINCENT Q. WILLIAMS
  CitationNumber DateScraped CaseStatus      DefenseAttorney
0      2A0365587  2023-06-09       OPEN  VINCENT Q. WILLIAMS
  CitationNumber DateScraped CaseStatus         DefenseAttorney
0      2A0530631  2023-06-09       OPEN  MITCHELL REUBEN GREENE
  CitationNumber DateScraped CaseStatus DefenseAttorney
0      9Z0828346  2023-06-09    DEFAULT             NaN
  CitationNumber DateScraped Cas

In [22]:
case_data = case_details[["CitationNumber", "DateScraped", "CaseStatus", "DefenseAttorney"]].copy()
case_data.loc[:,"CaseStatusChange"] = 0
case_data.loc[:,"DefenseAttorneyChange"] = 0
case_data_summary = case_data.groupby("CitationNumber")
#Compare changes in attorney name and case_status
for citation, data in case_data_summary:
    data_reindexed = pandas.DataFrame(data).reset_index(drop=True)
    casestatuses = data_reindexed["CaseStatus"].values
    casestatus0 = casestatuses[0]
    attorneys = data_reindexed["DefenseAttorney"].values
    attorney0 = attorneys[0]
    for i in range(len(data_reindexed)):
        if casestatuses[i] != casestatus0:
            casestatus0 = casestatuses[i]
            data_reindexed.at[i, "CaseStatusChange"] = True
        else:
            data_reindexed.at[i, "CaseStatusChange"] = False
        if attorneys[i] != attorney0 and not np.isnan(attorney0):
            attorney0 = attorneys[i]
            data_reindexed.at[i, "DefenseAttorneyChange"] = True
        else:
            data_reindexed.at[i, "DefenseAttorneyChange"] = False
    case_data.loc[data.index, "CaseStatusChange"] = data_reindexed["CaseStatusChange"].values
    case_data.loc[data.index, "DefenseAttorneyChange"] = data_reindexed["DefenseAttorneyChange"].values
    
case_data.sort_values(["CitationNumber", "DateScraped"], ascending=False).reset_index(drop=True)

,CitationNumber,DateScraped,CaseStatus,DefenseAttorney,CaseStatusChange,DefenseAttorneyChange
0,9Z0828346,2023-07-06,DEFAULT,NaN,False,False
1,9Z0828346,2023-07-05,DEFAULT,NaN,False,False
2,9Z0828346,2023-07-02,DEFAULT,NaN,False,False
3,9Z0828346,2023-06-28,DEFAULT,NaN,False,False
4,9Z0828346,2023-06-27,DEFAULT,NaN,False,False
5,9Z0828346,2023-06-09,DEFAULT,NaN,True,False
6,9Z0828346,2023-03-18,OPEN,NaN,False,False
7,73237039,2023-07-06,OPEN,NaN,False,False
8,73237039,2023-07-05,OPEN,NaN,False,False
9,73237039,2023-07-02,OPEN,NaN,False,False


In [31]:
charges = pandas.read_csv("../Data/charges.csv")
summary = charges.groupby(["CitationNumber", "DateScraped"])
# for citation_date, data in summary:
#     print(data)

In [32]:
charges_data = charges[["CitationNumber", "DateScraped", "Dispo"]].copy()
charges_data.loc[:,"DispoChange"] = 0
charges_data_summary = charges_data.groupby("CitationNumber")
for citation, data in charges_data_summary:
    data_reindexed = pandas.DataFrame(data).reset_index(drop=True)
    data_summary = data_reindexed.groupby("DateScraped")
    dispos_values = []
    for date, dispos in data_summary:
        dispos_reindexed = pandas.DataFrame(dispos).reset_index(drop=True)
        dispos_values.append(dispos_reindexed["Dispo"].values.tolist())
    #print(len(dispos_values)) -> should be the same for each group
    
    #Code to map f: [0,i]x[0,j] -> [0,nrow(data_reindexed)] ------------------------
    #Creates a dictionary that will allows us to find which entry (found by the index in the codomain)
    #    to store True/False in the data_reindexed array by using i and j
    i_values = []
    j_values = []

    for i in range(len(dispos_values)):
        for j in range(len(dispos_values[i])):
            i_values.append(i)
            j_values.append(j)

    index = {
        'i': i_values,
        'j': j_values
    }

    index['index'] = list(range(len(index['i'])))

    index = pandas.DataFrame(index)
    #-----------------------------------------------------------------------------
    
    dispo0 = dispos_values[0]
    for i in range(len(dispos_values)):
        for j in range(len(dispos_values[i])):
            data_index = index[(index["i"] == i) & (index["j"] == j)]["index"].values[0]
            if dispo0 == dispos_values[i]:
                #Only count it false if each group of dispos are *exactly* the same. i.e. a change from ['DF'] to ['DF','AM'] will have two True values in both rows
                if not (data_index > 0 and (data_reindexed.at[data_index-1, "DispoChange"] == True and (j > 0))):
                    data_reindexed.at[data_index, "DispoChange"] = False
                    
            else:
                data_reindexed.at[data_index, "DispoChange"] = True
                for k in range(len(dispos_values[i])):
                    data_reindexed.at[data_index+k, "DispoChange"] = True  
                dispo0 = dispos_values[i]
    print(data_reindexed)
                
    charges_data.loc[data.index, "DispoChange"] = data_reindexed["DispoChange"].values
    
charges_data.sort_values(["CitationNumber", "DateScraped"], ascending=False).reset_index(drop=True)

  CitationNumber DateScraped Dispo DispoChange
0     2100935015  2023-03-18    CN       False
1     2100935015  2023-06-09    CN       False
2     2100935015  2023-06-27    CN       False
3     2100935015  2023-06-28    CN       False
4     2100935015  2023-07-02    CN       False
5     2100935015  2023-07-05    CN       False
6     2100935015  2023-07-06    CN       False
  CitationNumber DateScraped Dispo DispoChange
0     2200192913  2023-03-18    CN       False
1     2200192913  2023-06-09    CN       False
2     2200192913  2023-06-27    CN       False
3     2200192913  2023-06-28    CN       False
4     2200192913  2023-07-02    CN       False
5     2200192913  2023-07-05    CN       False
6     2200192913  2023-07-06    CN       False
  CitationNumber DateScraped Dispo DispoChange
0     2300538791  2023-06-24    DF       False
1     2300538791  2023-06-26    DF       False
2     2300538791  2023-06-27    DF       False
3     2300538791  2023-06-28    DF       False
4     2300538

,CitationNumber,DateScraped,Dispo,DispoChange
0,9Z0828346,2023-07-06,DF,False
1,9Z0828346,2023-07-05,DF,False
2,9Z0828346,2023-07-02,DF,False
3,9Z0828346,2023-06-28,DF,False
4,9Z0828346,2023-06-27,DF,False
...,...,...,...,...
85,2100935015,2023-07-02,CN,False
86,2100935015,2023-06-28,CN,False
87,2100935015,2023-06-27,CN,False
88,2100935015,2023-06-09,CN,False


In [33]:
dispos = [[1], [3, 4], [5, 6], [7, 8]]

i_values = []
j_values = []

for i in range(len(dispos)):
    for j in range(len(dispos[i])):
        i_values.append(i)
        j_values.append(j)

index = {
    'i': i_values,
    'j': j_values
}

index['index'] = list(range(len(index['i'])))

index = pandas.DataFrame(index)

index[(index["i"] == 1) & (index["j"] == 1)]["index"].values[0]

2

In [34]:
import analysis
import datetime
o = analysis.obligations_summary()
c = analysis.case_details_summary()
ch = analysis.charges_summary()

C:\Users\Owner\Desktop\BYU\Research\Economics\Traffic Citations\analysis.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charges_data.loc[:,"DispoChange"] = 0


In [35]:
def getChanges(o,c,ch): #pass in dfs from obligation_summary, case_details_summary, and charges_summary
    obligation_citations = o.groupby("CitationNumber")
    case_citations = c.groupby("CitationNumber")
    charge_citations = ch.groupby("CitationNumber")
    ChangeTotals = {
        "CitationNumber": [],
        "TotalChanges": []
    }
    #Some tables might contain new or different citations, so we will just summarize the changes after aggregation
    for citation, data in obligation_citations:
        ChangeTotals["CitationNumber"].append(citation)
        data = pandas.DataFrame(data)
        ChangeTotals["TotalChanges"].append(np.sum(data["BalanceChange"].values.astype(int)) + np.sum(data["OriginalAmountChange"].values.astype(int)))
    for citation, data in case_citations:
        ChangeTotals["CitationNumber"].append(citation)
        data = pandas.DataFrame(data)
        ChangeTotals["TotalChanges"].append(np.sum(data["CaseStatusChange"].values.astype(int)) + np.sum(data["DefenseAttorneyChange"].values.astype(int)))
    for citation, data in charge_citations:
        ChangeTotals["CitationNumber"].append(citation)
        data = pandas.DataFrame(data)
        ChangeTotals["TotalChanges"].append(np.sum(data["DispoChange"].values.astype(int)))
    ChangeTotals = pandas.DataFrame(ChangeTotals).groupby("CitationNumber").sum().reset_index()
    return ChangeTotals.sort_values(["TotalChanges"])

In [36]:
changes = getChanges(o,c,ch)

In [38]:
#Add a date column to the changes table
changes["DateAnalyzed"] = [datetime.date.today()]*len(changes)
changes

,CitationNumber,TotalChanges,DateAnalyzed
0,2100935015,0,2023-07-13
1,2200192913,0,2023-07-13
2,2300538791,0,2023-07-13
3,2300585594,0,2023-07-13
4,2300665354,0,2023-07-13
5,2A0084843,0,2023-07-13
6,2A0365587,0,2023-07-13
8,73237039,2,2023-07-13
9,9Z0828346,3,2023-07-13
7,2A0530631,4,2023-07-13


In [39]:
import datetime
import numpy as np
import pandas
summary_files=["Obligations Summary.csv", "Case Details Summary.csv", "Charges Summary.csv"]
dir="Data"

changes = pandas.read_csv("../Data/Total Changes.csv")
dates = np.unique(changes["DateAnalyzed"].values)
#If the threshold doesn't return any matches, then just compare the previous group
current_date = datetime.datetime.strptime(changes["DateAnalyzed"].values[len(changes)-1], '%Y-%m-%d')
threshold_date = current_date-datetime.timedelta(days=1)

In [41]:
current_changes = changes[(changes["DateAnalyzed"] == str(current_date).split(' ')[0])]
past_changes = changes[(changes["DateAnalyzed"] == str(threshold_date).split(' ')[0])]

differences = {
    "CitationNumber": [],
    "IncreasedChanges": [],
    "NewCitation": [],
    "DeletedCitation": []
}

if len(past_changes) == 0: #This means that our threshold was a bad choice
    #Then just compare the next most previous scraped data
    past_changes = changes[(changes["DateAnalyzed"] == dates[len(dates)-2])]

for citation in current_changes["CitationNumber"]:
    row = current_changes[current_changes["CitationNumber"] == citation]
    #Because there should only be one entry for each unique citation, we can compare directly:
    
    past_changes_row = past_changes[past_changes["CitationNumber"] == citation]

    #If our new changes detect a newly added citation, then the current citation won't be in the past_changes table
    if len(past_changes_row) != 0:
        current_citation_changes = row["TotalChanges"].values[0]
        past_citation_changes = past_changes_row["TotalChanges"].values[0]

        difference = current_citation_changes != past_citation_changes

        if difference:
            differences["CitationNumber"].append(citation)
            differences["IncreasedChanges"].append(current_citation_changes-past_citation_changes)
            differences["NewCitation"].append(False)  
    else: #We will count it as a new change in the table
        differences["CitationNumber"].append(citation)
        differences["IncreasedChanges"].append(current_citation_changes)
        differences["NewCitation"].append(True)
        differences["DeletedCitation"].append(False)

#Now we will checked for citations that were there previously but not anymore
for citation in past_changes["CitationNumber"]:
    row = past_changes[past_changes["CitationNumber"] == citation]
    #See if there is a corresponding row in the most recent scrape
    row_exists = len(current_changes[current_changes["CitationNumber"] == citation]) != 0
    
    past_citation_changes = row["TotalChanges"].values[0]
    current_citation_changes = current_changes[current_changes["CitationNumber"] == citation]["TotalChanges"]
    
    if not row_exists:
        differences["CitationNumber"].append(citation)
        differences["IncreasedChanges"].append(-past_citation_changes)
        differences["NewCitation"].append(False)
        differences["DeletedCitation"].append(True)
    elif current_citation_changes.values[0] != past_citation_changes: #Only append false if the corresponding row has a change
        differences["DeletedCitation"].append(False)
    
differences = pandas.DataFrame(differences)

#Now we want to find where the changes took place:

In [42]:
changed_data_array = []
for i in range(len(summary_files)):
    summary_file = summary_files[i]
    summary_data = pandas.read_csv(f"{dir}/{summary_file}")

    changed_citations = summary_data[summary_data["CitationNumber"].isin(differences["CitationNumber"])]

    changed_citations_grouped = changed_citations.groupby(["DateScraped", "CitationNumber"])
    groupAppended = False
    nextIteration = False
    changedData = pandas.DataFrame()
    
    #Reversing the order is important when we detect for past changes.
    #The loop first finds the row that is changed and then it finds the next group.
    #If it is reversed, we can see what the data has changed from
    
    for group, data in reversed(list(changed_citations_grouped)):
        if not changedData.empty:
            nextIteration = True
        changeCols = []
        for col in data:
            if "Change" in col:
                changeCols.append(col)
        changed_data = data[(data[changeCols] == True).any(axis=1)]
        if len(changed_data) > 0:
            if changedData.empty:
                changedData = changed_data.copy()
            else:
                changedData = pandas.concat([changedData, changed_data])
            groupAppended = False
            nextIteration = True
        if nextIteration and not groupAppended and not changedData.empty:
            #Append the next group that doesn't have changes so we can see the changes that took place
            #We want to "Get to the bottom of it." i.e. We want to go back to the earliest date when there wasn't a change
            if len(data[(data[changeCols] == True).any(axis=1)]) <= 0:
                changedData = pandas.concat([changedData, data])
                groupAppended = True
                nextIteration = False
    #print(changedData)
    if not changedData.empty:
        changed_data_array.append(changedData)
        


In [37]:
len(changed_data_array)

0

In [38]:
import analysis
analysis.analyzeChanges()

(Empty DataFrame
 Columns: [CitationNumber, IncreasedChanges, NewCitation, DeletedCitation]
 Index: [],
 [])

In [74]:
(o.merge(c, on=["DateScraped", "CitationNumber"], how="left")).merge(ch, on=["DateScraped", "CitationNumber"], how="left").reset_index(drop=True)

,DateScraped,CitationNumber,TotalOriginalAmount,TotalBalance,BalanceDifference,BalanceChange,OriginalAmountChange,CaseStatus,DefenseAttorney,CaseStatusChange,DefenseAttorneyChange,Dispo,DispoChange
0,2023-07-06,9Z0828346,191.0,191.0,0.0,False,False,DEFAULT,NaN,False,False,DF,False
1,2023-07-05,9Z0828346,191.0,191.0,0.0,False,False,DEFAULT,NaN,False,False,DF,False
2,2023-07-02,9Z0828346,191.0,191.0,0.0,False,False,DEFAULT,NaN,False,False,DF,False
3,2023-06-28,9Z0828346,191.0,191.0,0.0,False,False,DEFAULT,NaN,False,False,DF,False
4,2023-06-27,9Z0828346,191.0,191.0,0.0,False,False,DEFAULT,NaN,False,False,DF,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2023-07-02,2100935015,139.0,0.0,139.0,False,False,NaN,NaN,NaN,NaN,CN,False
86,2023-06-28,2100935015,139.0,0.0,139.0,False,False,NaN,NaN,NaN,NaN,CN,False
87,2023-06-27,2100935015,139.0,0.0,139.0,False,False,NaN,NaN,NaN,NaN,CN,False
88,2023-06-09,2100935015,139.0,0.0,139.0,False,False,NaN,NaN,NaN,NaN,CN,False
